In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import measure, segmentation
from vis_utils import load_volume, VolumeVisualizer, ColorMapVisualizer
from scipy.ndimage import zoom
from skimage.morphology import skeletonize, skeletonize_3d
from skimage import filters, morphology

from scipy.ndimage.filters import convolve, correlate
from scipy import signal

In [2]:
source_dir = './data/'
files = list(sorted(glob.glob(source_dir + '/*/*.raw')))
list(enumerate(files))

[(0, './data/P12/P12_60um_1333x443x864.raw')]

In [3]:
%%time
volume = load_volume(files[0], scale=0.5)
# visualizer = VolumeVisualizer(volume, binary=False).visualize()

CPU times: user 1.36 s, sys: 429 ms, total: 1.79 s
Wall time: 2.61 s


## simple threshold segmentation

In [4]:
threshold = 70
mask_raw = volume > threshold

# VolumeVisualizer(mask_raw).visualize()

In [5]:
def get_main_regions(binary_mask, min_size=10_000, connectivity=3):
    labeled = measure.label(binary_mask, connectivity=connectivity)
    region_props = measure.regionprops(labeled)
    
    main_regions_masks = []
    
    for props in region_props:
        if props.area >= min_size:
            main_regions_masks.append((props.filled_image, props.bbox))
            
    return main_regions_masks

def merge_masks(masks, img_shape):
    result_mask = np.zeros(img_shape, dtype=np.uint8)
    for mask, bbox in masks:
        min1, min2, min3, max1, max2, max3 = bbox
        result_mask[min1:max1, min2:max2, min3:max3] += mask.astype(np.uint8)
        
    return result_mask

In [6]:
%%time
main_regions_masks = get_main_regions(mask_raw, min_size=5_000, connectivity=1)

CPU times: user 8.74 s, sys: 736 ms, total: 9.48 s
Wall time: 9.49 s


In [7]:
mask = main_regions_masks[0][0]
# VolumeVisualizer(mask).visualize()

## utility functions

In [8]:
def spherical_kernel(outer_radius, thickness=1, filled=True):    
    outer_sphere = morphology.ball(radius=outer_radius)
    if filled:
        return outer_sphere
    
    inner_radius = outer_radius - thickness
    inner_sphere = morphology.ball(radius=inner_radius)
    
    begin = outer_radius - inner_radius
    end = begin + inner_sphere.shape[0]
    outer_sphere[begin:end, begin:end, begin:end] -= inner_sphere
    return outer_sphere

def convolve_with_ball(mask, ball_radius, dtype=np.uint16):
    kernel = spherical_kernel(ball_radius, filled=True)
    return signal.convolve(mask.astype(dtype), kernel.astype(dtype), mode='same')

def get_arterial_regions(conv_img, lower_hyst_fraction, upper_hyst_fraction):
    lower_hyst_value = lower_hyst_fraction * conv_img.max()
    upper_hyst_value = upper_hyst_fraction * conv_img.max()
    return filters.apply_hysteresis_threshold(conv_img, lower_hyst_value, upper_hyst_value)

def reconstruct_from_skeleton(skeleton, ball_radius):    
    mask = np.zeros(skeleton.shape, dtype=np.uint8)
    mask = np.pad(mask, ball_radius)
    
    kernel = spherical_kernel(ball_radius, filled=True)
    central_points = np.argwhere(skeleton == 1)
    
    for central_point in central_points:
        start_corner = tuple(central_point)
        end_corner = tuple(central_point + 2*ball_radius + 1)
        
        start1, start2, start3 = start_corner
        end1, end2, end3 = end_corner
        
        mask_slice = mask[start1:end1, start2:end2, start3:end3]
        mask_slice[:] = np.logical_or(mask_slice, kernel)
                
    return mask[ball_radius:-ball_radius, ball_radius:-ball_radius, ball_radius:-ball_radius]

# high level functions

def get_tree_core(tree_mask, kernel_radius, max_fraction):
    convolved_mask = convolve_with_ball(tree_mask, kernel_radius)
    core_voxels = convolved_mask > max_fraction * convolved_mask.max()
    core_skeleton = skeletonize_3d(core_voxels.astype(np.uint8))
    core_reconstruction = reconstruct_from_skeleton(core_skeleton, kernel_radius)
    
    return core_reconstruction

def expand_tree_reconstruction(tree_mask, reconstruction, kernel_radius, max_fraction):
    convolved_mask = convolve_with_ball(tree_mask, kernel_radius)
    
    kernel_vol = spherical_kernel(kernel_radius).sum()
    threshold_value = int(max_fraction * kernel_vol)
    
    # set current reconstruction to infinity
    convolved_mask_with_huge_core = convolved_mask + reconstruction * (kernel_vol + 2)
        
        
    expanded_rec = filters.apply_hysteresis_threshold(convolved_mask_with_huge_core, threshold_value, kernel_vol + 5)
    expansion = expanded_rec - reconstruction
    
    convolved_mask_with_huge_expansion = convolved_mask + expansion * (kernel_vol + 2)
    expanded_expansion = filters.apply_hysteresis_threshold(convolved_mask_with_huge_expansion, threshold_value, kernel_vol + 5)
    
    ee_skeleton = skeletonize_3d(expanded_expansion.astype(np.uint8))
    ee_reconstruction = reconstruct_from_skeleton(ee_skeleton, kernel_radius)
    
    return ee_reconstruction

In [9]:
%%time
core_rec = get_tree_core(mask, 15, 0.95)
# VolumeVisualizer(np.logical_or(core_rec, mask)).visualize()
VolumeVisualizer(core_rec).visualize()

CPU times: user 11.1 s, sys: 4.37 s, total: 15.5 s
Wall time: 19.4 s


In [10]:
%%time
ee = expand_tree_reconstruction(mask, core_rec, kernel_radius=10, max_fraction=0.8)
VolumeVisualizer(ee).visualize()

CPU times: user 12.8 s, sys: 3.64 s, total: 16.5 s
Wall time: 21.4 s


In [11]:
%%time

rec = get_tree_core(mask, 15, 0.95)
total_rec = rec.copy().astype(np.uint8)
new_rec = rec.copy()
print('core is nice')

for i, kernel_radius in enumerate([10, 8, 7, 6, 5, 5, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2]):
    new_rec = expand_tree_reconstruction(mask, new_rec, kernel_radius=kernel_radius, max_fraction=0.5)
    rec = np.logical_or(rec, new_rec).astype(np.uint8)
    
    just_expansion = new_rec.copy()
    just_expansion[total_rec > 0] = 0
    total_rec += just_expansion * (i + 2)
    
    print('iter for', kernel_radius, 'ended successfully XD')
    

core is nice
iter for 10 ended successfully XD
iter for 8 ended successfully XD
iter for 7 ended successfully XD
iter for 6 ended successfully XD
iter for 5 ended successfully XD
iter for 5 ended successfully XD
iter for 4 ended successfully XD
iter for 4 ended successfully XD
iter for 4 ended successfully XD
iter for 3 ended successfully XD
iter for 3 ended successfully XD
iter for 3 ended successfully XD
iter for 3 ended successfully XD
iter for 2 ended successfully XD
iter for 2 ended successfully XD
iter for 2 ended successfully XD
CPU times: user 3min 13s, sys: 34.4 s, total: 3min 47s
Wall time: 3min 48s


In [19]:
VolumeVisualizer(rec).visualize()

In [20]:
ColorMapVisualizer(total_rec).visualize(interactive=False)

In [21]:
ColorMapVisualizer(mask + total_rec).visualize(interactive=True)

In [15]:
reconstruction_skeleton = skeletonize_3d(rec)
VolumeVisualizer(reconstruction_skeleton).visualize()

In [22]:
mask_with_no_skeleton = mask.copy()
mask_with_no_skeleton[reconstruction_skeleton == 1] = 0

VolumeVisualizer(reconstruction_skeleton.astype(np.uint8) * 2 + mask_with_no_skeleton, binary=False).visualize(primary_color=(1,1,1))

In [17]:
VolumeVisualizer(reconstruction_skeleton).visualize()

In [23]:
mask_skel = skeletonize_3d(mask.astype(np.uint8))

mask_with_no_skeleton = mask.copy()
mask_with_no_skeleton[mask_skel == 1] = 0

VolumeVisualizer(mask_skel.astype(np.uint8) * 2 + mask_with_no_skeleton, binary=False).visualize(primary_color=(1,1,1))